We create a Dask client

In [9]:
import dask
from dask.distributed import Client
client = Client(processes=False)
client

/opt/conda/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38771 instead
  http_address["port"], self.http_server.port


Client Scheduler: inproc://10.64.13.2/441/8 Dashboard: http://10.64.13.2:38771/status,Cluster Workers: 1 Cores: 2 Memory: 4.29 GB


In [33]:
import xarray as xa
def calc_stats(url):
    bf = xa.open_rasterio(url, chunks={'band': 1, 'x': 1024, 'y': 1024})
    mean_band= bf.mean()
    return mean_band

Test it locally 

In [34]:
url = 'http://landsat-pds.s3.amazonaws.com/c1/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1/'
redband = url+'LC08_L1TP_227065_20200608_20200626_01_T1_B{}.TIF'.format(4)

redband

'http://landsat-pds.s3.amazonaws.com/c1/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1/LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF'

In [35]:
mean=calc_stats(redband)

We will use client.submit to execute the computation on a distributed worker:

In [39]:
future = client.submit(calc_stats, redband)

In [40]:
future

<Future: finished, type: xarray.DataArray, key: calc_stats-67e3da04957bf0feab864bb9d4e244ef>

Lets do two files

We are now ready to get mean across many files using distributed workers. We can use map operation which is non-blocking, and one can continue to work in the Python shell/notebook while the computations are running.

In [42]:
b4 = url+'LC08_L1TP_227065_20200608_20200626_01_T1_B{}.TIF'.format(4)
b5 = url+'LC08_L1TP_227065_20200608_20200626_01_T1_B{}.TIF'.format(5)
b6 = url+'LC08_L1TP_227065_20200608_20200626_01_T1_B{}.TIF'.format(6)
filenames=[b4,b5,b6]

In [43]:
futures = client.map(calc_stats, filenames)

In [44]:
len(futures)

3

In [48]:
futures[:3]

[<Future: finished, type: xarray.DataArray, key: calc_stats-67e3da04957bf0feab864bb9d4e244ef>,
 <Future: finished, type: xarray.DataArray, key: calc_stats-4b33172b3ddf49750d64564c10c48759>,
 <Future: finished, type: xarray.DataArray, key: calc_stats-9e21c1bd6aa5f8d09d95c2f169a4cd2c>]

In [46]:
from distributed import progress

In [49]:
#progress(futures)